In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
import inception_preprocessing
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import os
import time
import subprocess

slim = tf.contrib.slim

In [2]:
# prepare label
place = ['house', 'lab', 'office']
place = place[0:1]
num = ['4', '5', '6']
# num = ['1']
# num = num[0:1]
kind = ['left', 'right']
# kind = kind[0:1]

path = '../labels/'

label = np.array([])

for p in place:
    lab_list = os.listdir(path + p)
    for lab in lab_list:
        ali = lab
        if 'obj' in ali:
            ali = ali.replace('.npy', '')
            for k in kind:
                if k in ali:
                    if p == 'lab':
                        for n in range(5, 9):
                            if str(n) in ali:
                                if len(label) == 0:
                                    label = np.load(path + p + '/' + lab)
                                else:
                                    label = np.concatenate([label, np.load(path + p + '/' + lab)])
                    else:
                        for n in num:
                            if n in ali:
                                if len(label) == 0:
                                    label = np.load(path + p + '/' + lab)
                                else:
                                    label = np.concatenate([label, np.load(path + p + '/' + lab)])
label = label.astype(int)
# label = label[0:2]
print(len(label))
img_num = len(label)

5292


In [4]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')

log_dir = '../log'
log_eval = '../log_eval_test'
batch_size = 32
num_epochs = 1
checkpoint_file = tf.train.latest_checkpoint(log_dir)

#State the number of classes to predict:
obj = { 'free':0,
        'computer':1,
        'cellphone':2,
        'coin':3,
        'ruler':4,
        'thermos-bottle':5,
        'whiteboard-pen':6,
        'whiteboard-eraser':7,
        'pen':8,
        'cup':9,
        'remote-control-TV':10,
        'remote-control-AC':11,
        'switch':12,
        'windows':13,
        'fridge':14,
        'cupboard':15,
        'water-tap':16,
        'toy':17,
        'kettle':18,
        'bottle':19,
        'cookie':20,
        'book':21,
        'magnet':22,
        'lamp-switch':23}

num_classes = len(obj)
obj_by_value = {}

for key in obj.keys():
    obj_by_value[obj[key]] = key

#Create log_dir for evaluation information
if not os.path.exists(log_eval):
    os.mkdir(log_eval)

#Just construct the graph from scratch again
with tf.Graph().as_default() as graph:
    tf.logging.set_verbosity(tf.logging.INFO)

    #Create some information about the training steps
    num_batches_per_epoch = int(img_num / batch_size)
    num_steps_per_epoch = num_batches_per_epoch
    
    # prepare image FIFOQueue
    height = 299
    width = 299
    num_threads = 4

    image_names = tf.train.match_filenames_once('../frames/test/house/*/*hand/Image*.png')
    image_queue = tf.train.string_input_producer(image_names, shuffle = False)

    image_reader = tf.WholeFileReader()
    _, image_value = image_reader.read(image_queue)

    image_tf = tf.image.decode_png(image_value, channels = 3)
    image_tf = tf.image.resize_images(image_tf, [height, width])
    image_tf.set_shape((height, width, 3))
    image_tf = tf.subtract(tf.multiply(2.0, tf.divide(image_tf, 255.0)), 0.5)
#     image_tf = inception_preprocessing.preprocess_image(image_tf, height, width, is_training = True)

    # prepare label FIFOQueue
    label_tf = tf.convert_to_tensor(label, dtype = tf.int32)
    label_queue = tf.train.slice_input_producer([label_tf], shuffle = False)
    
    # prepare original image for visualize
    raw_image = tf.image.decode_png(image_value, channels = 3)
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)
    
    # creating a batch of images and labels
    batch_image, batch_raw_image, batch_label = tf.train.batch([[image_tf], [raw_image], label_queue], 
                                                                batch_size = batch_size, 
                                                                num_threads = num_threads,
                                                                enqueue_many = True,
                                                                allow_smaller_final_batch = True)
    
    
    #Now create the inference model but set is_training=False
#     with slim.arg_scope(inception_resnet_v2_arg_scope()):
#         logits, end_points = inception_resnet_v2(batch_image, num_classes = num_classes, is_training = True)

#     #get all the variables to restore from the checkpoint file and create the saver function to restore
#     variables_to_restore = slim.get_variables_to_restore()
#     saver = tf.train.Saver(variables_to_restore)
#     def restore_fn(sess):
#         return saver.restore(sess, checkpoint_file)

#     #Just define the metrics to track without the loss or whatsoever
#     predictions = tf.argmax(end_points['Predictions'], 1)
#     accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, batch_label)
#     metrics_op = tf.group(accuracy_update)

#     #Create the global step and an increment op for monitoring
#     global_step = get_or_create_global_step()
#     global_step_op = tf.assign(global_step, global_step + 1) #no apply_gradient method so manually increasing the global_step

#     #Create a evaluation step function
#     def eval_step(sess, metrics_op, global_step):
#         start_time = time.time()
#         _, global_step_count, accuracy_value, prediction = sess.run([metrics_op, global_step_op, accuracy, predictions])
#         time_elapsed = time.time() - start_time

#         #Log some information
#         logging.info('Global Step %s: Streaming Accuracy: %.4f (%.2f sec/step)', global_step_count, accuracy_value, time_elapsed)
#         if global_step % 10 == 0:
#             for i in range(batch_size):
#                 logging.info('Current Streaming Predictions: %d', prediction[i])
#         return accuracy_value

#     #Define some scalar quantities to monitor
#     tf.summary.scalar('Validation_Accuracy', accuracy)
#     my_summary_op = tf.summary.merge_all()

    #Get your supervisor
    sv = tf.train.Supervisor(logdir = log_eval, summary_op = None, saver = None, init_fn = restore_fn)

    #Now we are ready to run in one session
    with sv.managed_session() as sess:
        '''
        accuracry section
        '''
#         for step in range(num_steps_per_epoch * num_epochs):
#             sess.run(sv.global_step)
#             #Compute summaries every 10 steps and continue evaluating
#             if step % 10 == 0:
#                 eval_step(sess, metrics_op = metrics_op, global_step = sv.global_step)
#                 summaries = sess.run(my_summary_op)
#                 sv.summary_computed(sess, summaries)

#             #Otherwise just run as per normal
#             else:
#                 eval_step(sess, metrics_op = metrics_op, global_step = sv.global_step)

#         #At the end of all the evaluation, show the final accuracy
#         logging.info('Final Streaming Accuracy: %.4f', sess.run(accuracy))
    
        '''
        Now we want to visualize the last batch's images just to see what our model has predicted
        
        '''
#         pred_raw_image, pred_label, predicts = sess.run([batch_raw_image, batch_label, predictions])
#         for i in range(batch_size):
#             image, label, predict = pred_raw_image[i], pred_label[i], predicts[i]
#             prediction_name, label_name = obj_by_value[int(predict)], obj_by_value[int(label)]
#             text = 'Prediction: %s \n Ground Truth: %s' %(prediction_name, label_name)
#             print(text)
        '''
        test label and image are right
        
        '''
        for step in range(200):
            pred_raw_image, pred_label = sess.run([batch_raw_image, batch_label])
            for i in range(batch_size):
                img_plot = plt.imshow(pred_raw_image[i])

                #Set up the plot and hide axes
                plt.title(obj_by_value[int(pred_label[i])])
                img_plot.axes.get_yaxis().set_ticks([])
                img_plot.axes.get_xaxis().set_ticks([])
                plt.show()

#         logging.info('Model evaluation has completed! Visit TensorBoard for more information regarding your evaluation.')

INFO:tensorflow:Restoring parameters from None
INFO:tensorflow:Error reported to Coordinator: <class 'TypeError'>, Cannot interpret feed_dict key as Tensor: The name 'save/Const:0' refers to a Tensor which does not exist. The operation, 'save/Const', does not exist in the graph.


TypeError: Cannot interpret feed_dict key as Tensor: The name 'save/Const:0' refers to a Tensor which does not exist. The operation, 'save/Const', does not exist in the graph.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
import inception_preprocessing
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import os
import time
import subprocess

slim = tf.contrib.slim
img_num = 0

# loading labels data 
place = ['house', 'lab', 'office']
# house = house[0:1]
num = ['1', '2', '3']
num = num[0:1]
kind = ['left', 'right']
kind = kind[0:1]

path = '../labels/'

label = np.array([])

for p in place:
    lab_list = os.listdir(path + p)
    for lab in lab_list:
        ali = lab
        if 'obj' in ali:
            ali = ali.replace('.npy', '')
            for k in kind:
                if k in ali:
                    if p == 'lab':
                        for n in ['1', '2', '3', '4']:
                            if n in ali:
                                if len(label) == 0:
                                    label = np.load(path + p + '/' + lab)
                                else:
                                    label = np.concatenate([label, np.load(path + p + '/' + lab)])
                    else:
                        for n in num:
                            if n in ali:
                                if len(label) == 0:
                                    label = np.load(path + p + '/' + lab)
                                else:
                                    label = np.concatenate([label, np.load(path + p + '/' + lab)])
label = label.astype(int)
print('label data loaded, and the number of labels is:', len(label))
img_num = len(label)

#State where your log file is at. If it doesn't exist, create it.
log_dir = '../log'

#State where your checkpoint file is
checkpoint_file = '../inception_resnet_v2_2016_08_30.ckpt'

#State the number of classes to predict:
obj = { 'free':0,
        'computer':1,
        'cellphone':2,
        'coin':3,
        'ruler':4,
        'thermos-bottle':5,
        'whiteboard-pen':6,
        'whiteboard-eraser':7,
        'pen':8,
        'cup':9,
        'remote-control-TV':10,
        'remote-control-AC':11,
        'switch':12,
        'windows':13,
        'fridge':14,
        'cupboard':15,
        'water-tap':16,
        'toy':17,
        'kettle':18,
        'bottle':19,
        'cookie':20,
        'book':21,
        'magnet':22,
        'lamp-switch':23}

num_classes = len(obj)
num_epochs = 70
batch_size = 16
initial_learning_rate = 0.001
learning_rate_decay_factor = 0.8
num_epochs_before_decay = 2

if not os.path.exists(log_dir):
    os.mkdir(log_dir)

init_op = (tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init_op)

with tf.Graph().as_default() as graph:
    tf.logging.set_verbosity(tf.logging.INFO)

    num_batches_per_epoch = int(img_num / batch_size)
    print("num_batches_per_epoch:", num_batches_per_epoch)
    num_steps_per_epoch = num_batches_per_epoch
    decay_steps = int(num_epochs_before_decay * num_steps_per_epoch)

    # prepare image FIFOQueue
    height = 299
    width = 299
    num_threads = 4

    image_names = tf.train.match_filenames_once('../frames/train/*/*/*hand/Image*.png')
    image_queue = tf.train.string_input_producer(image_names, shuffle = False)

    image_reader = tf.WholeFileReader()
    _, image_value = image_reader.read(image_queue)

    image_tf = tf.image.decode_png(image_value, channels = 3)
    image_tf = tf.image.resize_images(image_tf, [height, width])
    image_tf.set_shape((height, width, 3))
    # image_tf = 2 * (image_tf / 255.0) - 1.0
    image_tf = tf.subtract(tf.multiply(2.0, tf.divide(image_tf, 255.0)), 1.0)
    # image_tf = inception_preprocessing.preprocess_image(image_tf, height, width, is_training = True)

    # prepare label FIFOQueue
    label_tf = tf.convert_to_tensor(label, dtype = tf.int32)
    label_queue = tf.train.slice_input_producer([label_tf], shuffle = False)

    # creating a batch of images and labels
    batch_image, batch_label = tf.train.batch([[image_tf], label_queue], 
                                            batch_size = batch_size, 
                                            num_threads = num_threads,
                                            capacity = num_threads * batch_size,
                                            enqueue_many = True,
                                            allow_smaller_final_batch = True)
    
    #Create the model inference
    with slim.arg_scope(inception_resnet_v2_arg_scope()):
        logits, end_points = inception_resnet_v2(batch_image, 
                                                 num_classes = num_classes, 
                                                 is_training = True)

    #Define the scopes that you want to exclude for restoration
    exclude = ['InceptionResnetV2/AuxLogits',
               'InceptionResnetV2/Logits']
    variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
    one_hot_labels = slim.one_hot_encoding(batch_label, 
                                           num_classes)
    loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, 
                                           logits = logits)

    regularization_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    all_losses = [loss] + regularization_losses
    total_loss = tf.add_n(all_losses, name='total_loss')
    # total_loss = tf.losses.get_total_loss()

    global_step = get_or_create_global_step()

    lr = tf.train.exponential_decay(
        learning_rate = initial_learning_rate,
        global_step = global_step,
        decay_steps = decay_steps,
        decay_rate = learning_rate_decay_factor,
        staircase = True)
    optimizer = tf.train.AdamOptimizer(learning_rate = lr)

    # get variables to be trained
    variables_to_train = []
    with tf.Session() as sess:
        for v in tf.trainable_variables():
            variables_to_train += [v]
   

label data loaded, and the number of labels is: 4056
num_batches_per_epoch: 253


In [5]:
for i, v in enumerate(variables_to_train):
    print(i, v)

0 <tf.Variable 'InceptionResnetV2/Conv2d_1a_3x3/weights:0' shape=(3, 3, 3, 32) dtype=float32_ref>
1 <tf.Variable 'InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>
2 <tf.Variable 'InceptionResnetV2/Conv2d_2a_3x3/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>
3 <tf.Variable 'InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>
4 <tf.Variable 'InceptionResnetV2/Conv2d_2b_3x3/weights:0' shape=(3, 3, 32, 64) dtype=float32_ref>
5 <tf.Variable 'InceptionResnetV2/Conv2d_2b_3x3/BatchNorm/beta:0' shape=(64,) dtype=float32_ref>
6 <tf.Variable 'InceptionResnetV2/Conv2d_3b_1x1/weights:0' shape=(1, 1, 64, 80) dtype=float32_ref>
7 <tf.Variable 'InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/beta:0' shape=(80,) dtype=float32_ref>
8 <tf.Variable 'InceptionResnetV2/Conv2d_4a_3x3/weights:0' shape=(3, 3, 80, 192) dtype=float32_ref>
9 <tf.Variable 'InceptionResnetV2/Conv2d_4a_3x3/BatchNorm/beta:0' shape=(192,) dtype=float32_ref>
10 <tf.Variable 'In

In [9]:
va = [variables_to_train[372:378], variables_to_train[494:496]]
print(va)

[[<tf.Variable 'InceptionResnetV2/AuxLogits/Conv2d_1b_1x1/weights:0' shape=(1, 1, 1088, 128) dtype=float32_ref>, <tf.Variable 'InceptionResnetV2/AuxLogits/Conv2d_1b_1x1/BatchNorm/beta:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'InceptionResnetV2/AuxLogits/Conv2d_2a_5x5/weights:0' shape=(5, 5, 128, 768) dtype=float32_ref>, <tf.Variable 'InceptionResnetV2/AuxLogits/Conv2d_2a_5x5/BatchNorm/beta:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'InceptionResnetV2/AuxLogits/Logits/weights:0' shape=(768, 24) dtype=float32_ref>, <tf.Variable 'InceptionResnetV2/AuxLogits/Logits/biases:0' shape=(24,) dtype=float32_ref>], [<tf.Variable 'InceptionResnetV2/Logits/Logits/weights:0' shape=(1536, 24) dtype=float32_ref>, <tf.Variable 'InceptionResnetV2/Logits/Logits/biases:0' shape=(24,) dtype=float32_ref>]]
